<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/nilm_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#What if we do under-sampling and put zero as a class in the RNN?
import io
from google.colab import files
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier as gbdt

In [0]:
'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

In [0]:
#uploaded = files.upload()
'''Load data'''
#WHE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_WHE.csv']))[['unix_ts','P']]
WHE_data = pd.read_csv('Electricity_WHE.csv')[['unix_ts','P']]
#uploaded = files.upload()
#CWE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_CWE.csv']))['P']
CDE_data = pd.read_csv('Electricity_CDE.csv')['P']

In [5]:
window_size=10
'''Using sliding window to build training examples'''
time_data = create_dataset(WHE_data['P'].as_matrix(), window_size)

WHE_data['datetime'] = pd.to_datetime(WHE_data['unix_ts'],unit='s')
WHE_data = WHE_data.drop(['unix_ts'],axis=1)

WHE_data['month'] = WHE_data['datetime'].dt.month
WHE_data['hour'] = WHE_data['datetime'].dt.hour
WHE_data['weekday'] = WHE_data['datetime'].dt.dayofweek
WHE_data['quarter'] = WHE_data['datetime'].dt.quarter
data_month = WHE_data['month'].as_matrix()[window_size-1:].reshape([-1,1])
data_hour = WHE_data['hour'].as_matrix()[window_size-1:].reshape([-1,1])
data_weekday = WHE_data['weekday'].as_matrix()[window_size-1:].reshape([-1,1])
dataX = np.hstack([time_data,data_month,data_hour,data_weekday])

print(1)
#get y for the single load
cde_Y = CDE_data[window_size-1:]
cdeY=cde_Y>0

X_train, X_test, y_train, y_test = train_test_split(dataX, cdeY, test_size=0.2, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_test = scaler.transform(X_test)

1


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.001               # learning rate

#hyperparameters
batch_size=512


In [0]:
'''Neural Network Model Definition'''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, 13])
tf_y = tf.placeholder(tf.float32, [None, 1])

l1 = tf.layers.dense(tf_x,13,activation=tf.nn.leaky_relu)
l2 = tf.layers.dense(l1,32,activation=tf.nn.leaky_relu)
l3 = tf.layers.dense(l2,64,activation=tf.nn.leaky_relu)
l4 = tf.layers.dense(l3,32,activation=tf.nn.leaky_relu)
l5 = tf.layers.dense(l4,16,activation=tf.nn.leaky_relu)
l6 = tf.layers.dense(l5,8,activation=tf.nn.leaky_relu)
l7 = tf.layers.dense(l6,4,activation=tf.nn.sigmoid)

pred = tf.layers.dense(l7,1,activation=None)
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_sum(- tf_y * tf.log(pred) - (1 - tf_y) * tf.log(1-pred), 1)
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.metrics.accuracy(          # return (acc, update_op), and create 2 local variables
    labels=tf.argmax(tf_y, axis=1), predictions=tf.argmax(pred, axis=1),)[1]

'''
gvs = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
train_op = optimizer.apply_gradients(capped_gvs) 
'''
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
sess = tf.Session()
sess.run(init_op)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("logss/", sess.graph) # tensorflow >=0.12
saver = tf.train.Saver()

In [16]:
batch_num = len(X_train)//batch_size
for j in range(0,2000):
    print('###epoch: '+str(j)+'###')
    for i in range(0,batch_num+1):
        if(i!=batch_num):
            batch_X = X_train[i*batch_size:(i+1)*batch_size,]
            batch_Y = y_train[i*batch_size:(i+1)*batch_size,].reshape([-1,1])
        else: 
            batch_X = X_train[-batch_size:]
            batch_Y = y_train[-batch_size:,].reshape([-1,1])
        sess.run(train_op,{tf_x:batch_X,tf_y:batch_Y})
        cost_ = sess.run(loss, {tf_x: batch_X, tf_y:batch_Y})
        acc_train = sess.run(accuracy,{tf_x: batch_X, tf_y:batch_Y})
        acc_test = sess.run(accuracy,feed_dict={tf_x: X_test, tf_y:y_test.reshape([-1,1])})
        if i%10 == 0:
            rs = sess.run(merged,feed_dict={tf_x: batch_X, tf_y:batch_Y})
            pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_Y})
            writer.add_summary(rs, i+j*batch_num)
            print('###Batch: '+str(i)+': train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
            print('Test Acc=%.2f'% acc_test)
            if acc_train>=0.995:
              save_path = saver.save(sess, "my_net/save_net_rnn.ckpt")
sess.close()

###epoch: 0###


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


###Batch: 0: train loss= nan, Acc=1.00
Test Acc=1.00
###Batch: 10: train loss= nan, Acc=1.00
Test Acc=1.00
###Batch: 20: train loss= nan, Acc=1.00
Test Acc=1.00
###Batch: 30: train loss= nan, Acc=1.00
Test Acc=1.00
###Batch: 40: train loss= nan, Acc=1.00
Test Acc=1.00


KeyboardInterrupt: ignored

In [6]:
batch_size=50000
model_list=[]
batch_num = len(X_train)//batch_size
print(batch_num)
for i in range(0,batch_num+1):
  if(i!=batch_num):
      batch_X = X_train[i*batch_size:(i+1)*batch_size,]
      batch_Y = y_train[i*batch_size:(i+1)*batch_size,]
  else: 
      batch_X = X_train[-batch_size:]
      batch_Y = y_train[-batch_size:,]
  
  print('###Build Model: '+str(i+1)+'###')
  model = SVC(class_weight='balanced',gamma='scale')
  print('pre train')
  model.fit(batch_X,batch_Y)
  print('train finished')
  model_list.append(model)

print('All the trainings have finished!')

16
###Build Model: 1###
pre train
train finished
###Build Model: 2###
pre train
train finished
###Build Model: 3###
pre train
train finished
###Build Model: 4###
pre train
train finished
###Build Model: 5###
pre train
train finished
###Build Model: 6###
pre train
train finished
###Build Model: 7###
pre train
train finished
###Build Model: 8###
pre train
train finished
###Build Model: 9###
pre train
train finished
###Build Model: 10###
pre train
train finished
###Build Model: 11###
pre train
train finished
###Build Model: 12###
pre train
train finished
###Build Model: 13###
pre train
train finished
###Build Model: 14###
pre train
train finished
###Build Model: 15###
pre train
train finished
###Build Model: 16###
pre train
train finished
###Build Model: 17###
pre train
train finished
All the trainings have finished!


In [7]:
models_output = np.zeros(y_test.shape)
for model in model_list:
  print(model.predict(X_test))
  models_output = models_output+model.predict(X_test)

[False False False ... False False False]


KeyboardInterrupt: ignored

In [42]:
models_output_vote = models_output>(batch_num/2)
print(models_output_vote)

[False False False ... False False False]


In [45]:
predict_acc = np.sum(models_output_vote==y_test)
print(predict_acc/y_test.shape)

[0.97970881]
